In [1]:
import tensorflow as tf
from keras.applications import MobileNetV2
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping



# import os
# from keras.utils import img_to_array, load_img
# import numpy as np
# from keras.preprocessing.image import ImageDataGenerator
# import pandas as pd
# from matplotlib import pyplot as plt

trainPath = r"C:\Users\gener\OneDrive\Documents\fruits360\fruits-360\train-small"
testPath = r"C:\Users\gener\OneDrive\Documents\fruits360\fruits-360\test-small"

batchSize = 64      # Reduce value if you have less GPU


In [ ]:
# WATCH VIDEO FOR IMAGE PRE-PROCESSING
# LOOK AT KAGGLE TO SEE HOW IMAGE PRE-PROCESSING IS DONE
# CHAT-GPT

model = Sequential()
mobileNet = MobileNetV2(input_shape=(100,100,3), include_top=False, classes=5)
for layer in mobileNet.layers:
    layer.trainable = False

model.add(mobileNet)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))




In [ ]:
# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

stepsPerEpoch = np.ceil(train_generator.samples / batchSize)
validationSteps = np.ceil(test_generator.samples / batchSize)

# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5, min_delta=0.001, restore_best_weights=True)

history = modelv2.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=50, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])

modelv2.save(r"C:\Users\gener\OneDrive\Documents\cv-food-app\fruits360Model-v2.h5") 